Import Modules

In [2]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
from config import google_api_key
import numpy as np
import requests
import json

Import files as dataframes

In [3]:
# Import files as DataFrames
fourteen_fiftenn_pd = pd.read_csv("../data/2014-15.csv")
seventeen_eighteen_pd = pd.read_csv("../data/2017-18.csv", encoding = "ISO-8859-1")
twenty_twentyone = pd.read_csv("../data/2020-21.csv", encoding = "ISO-8859-1")

Initial transformation to set up for API call for lats & lngs

In [4]:
# Rename columns to be uniform across dataframes
twenty_twentyone.rename(columns={"Facility name":"Facility Name"}, inplace=True)

In [5]:
# Remove rows of corporate total data
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Facility Name'] != 'Corporate Total']
twenty_twentyone = twenty_twentyone[twenty_twentyone['Facility Name'] != 'Corporate Total']

In [6]:
# Drop columns of un-needed data
fourteen_fiftenn_pd = fourteen_fiftenn_pd.drop(columns={"Type", "Scope 1 (t CO2-e)","Scope 2 (t CO2-e)","Total Emissions (t CO2-e)", "Emission  Intensity (t/Mwh)", "Grid Connected", "Grid", "Important Notes"})
seventeen_eighteen_pd = seventeen_eighteen_pd.drop(columns={"Type", "Total Scope 1 Emissions \n(t CO2-e)","Total Scope 2 Emissions \n(t CO2-e)","Total Emissions \n(t CO2-e)", "Emission Intensity \n(t CO2-e/ MWh)", "Grid Connected", "Grid"})
twenty_twentyone = twenty_twentyone.drop(columns={"Type", "Total scope 1 emissions \n(t CO2-e)","Total scope 2 emissions \n(t CO2-e)","Total emissions \n(t CO2-e)", "Emission intensity \n(t CO2-e/ MWh)", "Grid connected", "Grid"})

In [7]:
# Add year column for filtering in plots
fourteen_fiftenn_pd['Year'] = '2014-2015'
seventeen_eighteen_pd['Year'] = '2017-2018'
twenty_twentyone['Year'] = '2020-2021'

API call for lats & lngs

In [8]:
# create a params dict that will be updated with new city each iteration
params = {"key": google_api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fourteen_fiftenn_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fourteen_fiftenn_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fourteen_fiftenn_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fourteen_fiftenn_pd.loc[index, "Lat"] = "NaN"
        fourteen_fiftenn_pd.loc[index, "Lng"] = "NaN"

# Visualize to confirm lat lng appear
fourteen_fiftenn_pd.head()

,Reporting Entity,Facility Name,State,Electricity Production (GJ),Electricity Production (Mwh),Primary Fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,567719.0,157700.0,Wind,2014-2015,-34.6906,149.427
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,ACT,213115.0,59199.0,Solar,2014-2015,-35.4898,149.143
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,2461803.0,683834.0,Wind,2014-2015,-37.3946,143.639
4,AGL ENERGY LIMITED,Banimboola Hydro,VIC,137094.0,38082.0,Hydro,2014-2015,-36.5336,147.46
5,AGL ENERGY LIMITED,Bayswater Power Station,NSW,50275548.0,13965430.0,Black Coal,2014-2015,-32.3926,150.949


In [9]:
# create a params dict that will be updated with new city each iteration
params = {"key": google_api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in seventeen_eighteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        seventeen_eighteen_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        seventeen_eighteen_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        seventeen_eighteen_pd.loc[index, "Lat"] = "NaN"
        seventeen_eighteen_pd.loc[index, "Lng"] = "NaN"

seventeen_eighteen_pd.head()

,Reporting Entity,Facility Name,State,Electricity Production (GJ),Electricity Production (MWh),Primary Fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY. LTD.,Cathedral Rocks Wind Farm,SA,"550,950","153,042",Wind,2017-2018,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY. LTD.,Gunning Wind Farm,NSW,"601,740","167,150",Wind,2017-2018,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY. LTD.,Mt Gellibrand Wind Farm,VIC,"4,320","1,200",Wind,2017-2018,-38.2328,143.792
3,ACCIONA ENERGY OCEANIA PTY. LTD.,Waubra Wind Farm,VIC,"2,317,673","643,798",Wind,2017-2018,-37.3946,143.639
5,AGL ENERGY LIMITED,Banimboola Hydro,VIC,"11,089","3,080",Hydro,2017-2018,-36.5336,147.46


In [10]:
# create a params dict that will be updated with new city each iteration
params = {"key": google_api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twenty_twentyone.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location} + {state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twenty_twentyone.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twenty_twentyone.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twenty_twentyone.loc[index, "Lat"] = "NaN"
        twenty_twentyone.loc[index, "Lng"] = "NaN"

twenty_twentyone.head()

,Reporting entity,Facility Name,State,Electricity production (GJ),Electricity production (MWh),Primary fuel,Year,Lat,Lng
0,ACCIONA ENERGY OCEANIA PTY LTD,Cathedral Rocks Wind Farm,SA,"512,455","142,348",Wind,2020-2021,NaN,NaN
1,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,"575,522","159,867",Wind,2020-2021,-34.6906,149.427
2,ACCIONA ENERGY OCEANIA PTY LTD,Mt Gellibrand Wind Farm,VIC,"1,018,498","282,916",Wind,2020-2021,-38.2328,143.792
3,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,"2,339,624","649,896",Wind,2020-2021,-37.3946,143.639
5,ADANI RENEWABLE ASSET HOLDINGS PTY LTD,Adani Rugby Run Solar Farm,QLD,"587,856","163,293",Solar,2020-2021,-22.5752,144.085


In [11]:
# Drop rows which returned no coordinates for facilities
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Lat'] != 'NaN']

seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Lat'] != 'NaN']

twenty_twentyone = twenty_twentyone[twenty_twentyone['Lat'] != 'NaN']

Clean up the populated dataframes

In [12]:
fourteen_fifteen_transformed = fourteen_fiftenn_pd.rename(columns={"Reporting Entity ": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (Mwh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [13]:
seventeen_eighteen_transformed = seventeen_eighteen_pd.rename(columns={"Reporting Entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (MWh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [14]:
twenty_twentyone_transformed = twenty_twentyone.rename(columns={"Reporting entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity production (MWh)": "Electricity_Production_MWh",
                                                                   "Primary fuel": "Primary_Fuel"})

Categorize renewables

In [15]:
renewables = ['Hydro', 'Landfill Gas', 'Wind', 'Solar', 'Bagasse', 'Biofuel', 'Macadamia Nut Shells', 'Biogas']

fourteen_fifteen_transformed['Renewable'] = fourteen_fifteen_transformed.Primary_Fuel.isin(renewables)
seventeen_eighteen_transformed['Renewable'] = seventeen_eighteen_transformed.Primary_Fuel.isin(renewables)
twenty_twentyone_transformed['Renewable'] = twenty_twentyone_transformed.Primary_Fuel.isin(renewables)

Drop the index

In [20]:
# fourteen_fifteen_transformed.drop(columns=['index'], inplace=True)
# seventeen_eighteen_transformed.drop(columns=['index'], inplace=True)
# twenty_twentyone_transformed.drop(columns=['index'], inplace=True)

In [16]:
fourteen_fifteen_transformed.replace(',','', regex=True, inplace=True)

fourteen_fifteen_transformed["Electricity_Production_GJ"] = pd.to_numeric(fourteen_fifteen_transformed["Electricity_Production_GJ"])
fourteen_fifteen_transformed["Electricity_Production_MWh"] = pd.to_numeric(fourteen_fifteen_transformed["Electricity_Production_MWh"])
fourteen_fifteen_transformed["Lat"] = pd.to_numeric(fourteen_fifteen_transformed["Lat"])
fourteen_fifteen_transformed["Lng"] = pd.to_numeric(fourteen_fifteen_transformed["Lng"])

In [17]:
seventeen_eighteen_transformed.replace(',','', regex=True, inplace=True)

seventeen_eighteen_transformed["Electricity_Production_GJ"] = pd.to_numeric(seventeen_eighteen_transformed["Electricity_Production_GJ"])
seventeen_eighteen_transformed["Electricity_Production_MWh"] = pd.to_numeric(seventeen_eighteen_transformed["Electricity_Production_MWh"])
seventeen_eighteen_transformed["Lat"] = pd.to_numeric(seventeen_eighteen_transformed["Lat"])
seventeen_eighteen_transformed["Lng"] = pd.to_numeric(seventeen_eighteen_transformed["Lng"])


In [18]:
twenty_twentyone_transformed.replace(',','', regex=True, inplace=True)

twenty_twentyone_transformed["Electricity_Production_GJ"] = pd.to_numeric(twenty_twentyone_transformed["Electricity_Production_GJ"])
twenty_twentyone_transformed["Electricity_Production_MWh"] = pd.to_numeric(twenty_twentyone_transformed["Electricity_Production_MWh"])
twenty_twentyone_transformed["Lat"] = pd.to_numeric(twenty_twentyone_transformed["Lat"])
twenty_twentyone_transformed["Lng"] = pd.to_numeric(twenty_twentyone_transformed["Lng"])

In [19]:
twenty_twentyone_transformed.dtypes

Reporting_Entity               object
Facility_Name                  object
State                          object
Electricity_Production_GJ       int64
Electricity_Production_MWh      int64
Primary_Fuel                   object
Year                           object
Lat                           float64
Lng                           float64
Renewable                        bool
dtype: object

Export data as CSV

In [23]:
fourteen_fifteen_transformed.to_csv('../export_data/14-15.csv')
seventeen_eighteen_transformed.to_csv('../export_data/17-18.csv')
twenty_twentyone_transformed.to_csv('../export_data/20-21.csv')

In [20]:
combined_data = pd.concat([fourteen_fifteen_transformed, seventeen_eighteen_transformed, twenty_twentyone_transformed])
combined_data

,Reporting_Entity,Facility_Name,State,Electricity_Production_GJ,Electricity_Production_MWh,Primary_Fuel,Year,Lat,Lng,Renewable
0,ACCIONA ENERGY OCEANIA PTY LTD,Gunning Wind Farm,NSW,567719.0,157700.0,Wind,2014-2015,-34.690562,149.427009,True
1,ACCIONA ENERGY OCEANIA PTY LTD,Royalla Solar Farm,ACT,213115.0,59199.0,Solar,2014-2015,-35.489793,149.143002,True
2,ACCIONA ENERGY OCEANIA PTY LTD,Waubra Wind Farm,VIC,2461803.0,683834.0,Wind,2014-2015,-37.394639,143.638601,True
4,AGL ENERGY LIMITED,Banimboola Hydro,VIC,137094.0,38082.0,Hydro,2014-2015,-36.533577,147.460346,True
5,AGL ENERGY LIMITED,Bayswater Power Station,NSW,50275548.0,13965430.0,Black Coal,2014-2015,-32.392578,150.949051,False
...,...,...,...,...,...,...,...,...,...,...
645,WEL 1 PTY LTD,Wemen Solar Farm,VIC,666176.0,185049.0,Solar,2020-2021,-34.802576,142.544413,True
647,WHITE ROCK SOLAR FARM (HOLDING) PTY LTD,White Rock Solar Farm Pty Ltd,NSW,135137.0,37538.0,Solar,2020-2021,-29.762422,151.544743,True
649,WILD CATTLE HILL PTY LTD,Wild Cattle Hill Wind Farm,TAS,1562847.0,434124.0,Wind,2020-2021,-42.040906,146.808732,True
651,YARA AUSTRALIA PTY LTD,YPF UTILITIES PRODUCTION AND SUPPLY,WA,115658.0,32127.0,Gas,2020-2021,-27.672817,121.628310,False


In [21]:
combined_data.to_csv('../export_data/combined_plant_locations.csv')